In [7]:
import pandas as pd # Access the Pandas library, let's me utilise dataframes
import matplotlib as plt #Import matplotlib library, probably don't need it though
import numpy as np

In [8]:
# a is the claims
def delaytriangle(a):
    for i in range(0,a.shape[0]):
        for j in range(0,a.shape[1]):
            if a.isna().iloc[i][j] == True:
                a.iloc[i,j] = 0 #Replaces the nans with 0
                
    development_factors = []
    for j in range(a.shape[1]-1,1,-1):
        counter = 0
        counter_1 = 0 #reset counting definition
        for i in range(0,a.shape[0],1):
            if a.iloc[i,j] != 0:
                counter = counter + a.iloc[i,j] #Adds up the claims
                counter_1 = counter_1 + a.iloc[i,j-1] #adds the claims on the second right most column
        development_factors.append(counter/counter_1)#Provides the developement factors
        
    for i in range(a.shape[0]-1,-1,-1):
        k = len(development_factors)-1 
        for j in range(1,a.shape[1]):
            if j<a.shape[1] - 1 and a.iloc[i,j+1] == 0 :
                a.iloc[i,j+1] = a.iloc[i,j] * development_factors[k] #Multiplies the claims by developmenfactors to find next year's claim
            k = k - 1
            
    Total_Claims = 0
    for i in range(0,a.shape[0]):
        Total_Claims = Total_Claims + a.iloc[i,a.shape[1] - 1]
        
    Settled_Claims = 0   
    counter = a.shape[0]
    for i in range(0,a.shape[0]):
        Settled_Claims = Settled_Claims + a.iloc[i, counter]
        counter = counter - 1
        
    Required_Reserves = Total_Claims - Settled_Claims

    d_triangle = {'Table': a,
                 'Total_Claims': Total_Claims,
                 'Settled_Claims': Settled_Claims,
                 'Required_Reserves': Required_Reserves}
    return d_triangle

In [9]:
# a is the claims and b is the premiums
def bftriangle(a,b):
    for i in range(0,a.shape[0]):
        for j in range(0,a.shape[1]):
            if a.isna().iloc[i][j] == True:
                a.iloc[i,j] = 0 #Replaces the nans with 0
                
    development_factors = []
    for j in range(a.shape[1]-1,1,-1):
        counter = 0
        counter_1 = 0 #reset counting definition
        for i in range(0,a.shape[0],1):
            if a.iloc[i,j] != 0:
                counter = counter + a.iloc[i,j] #Adds up the claims
                counter_1 = counter_1 + a.iloc[i,j-1] #adds the claims on the second right most column
        development_factors.append(counter/counter_1)#Provides the developement factors
        
    for i in range(a.shape[0]-1,-1,-1):
        k = len(development_factors)-1 
        for j in range(1,a.shape[1]):
            if j<a.shape[1] - 1 and a.iloc[i,j+1] == 0 :
                a.iloc[i,j+1] = a.iloc[i,j] * development_factors[k] #Multiplies the claims by developmenfactors to find next year's claim
            k = k - 1
            
    Total_Claims = 0
    for i in range(0,a.shape[0]):
        Total_Claims = Total_Claims + a.iloc[i,a.shape[1] - 1]
        
    Settled_Claims = 0   
    counter = a.shape[0]
    for i in range(0,a.shape[0]):
        Settled_Claims = Settled_Claims + a.iloc[i, counter]
        counter = counter - 1
        
    Required_Reserves = Total_Claims - Settled_Claims
    
    ult = a.iloc[0, a.shape[1]-1]
    loss_ratio = ult/b.iloc[0,0]
    ult_list = prems * loss_ratio  #Produce the list of ultimate claims

    F = [1] #Making the cumulative development factors
    for i in range(0, len(development_factors)):
        F.append(F[i] * development_factors[i])

    newF = []
    for i in range(0, len(F)):
        newF.append(1 - 1/F[i])
        
    emerging = []
    for i in range(0,len(newF)):
        emerging.append(newF[i] * ult_list.iloc[i,0])

    ORR = 0 
    for i in range(0,len(emerging)):  #Calculate the overall reserve requirement
        ORR = ORR + emerging[i]

    bf_triangle = {'Table': a,
                'Total_Claims': Total_Claims,
                'Settled_Claims': Settled_Claims,
                'Required_Reserves': Required_Reserves,
                'Premiums': prems,
                'Ultimate_Claims': ult_list,
                'Emerging_Liabilities': emerging,
                 'Overall_Reserve_Requirement': ORR}
    return bf_triangle

In [10]:
delay_test = pd.read_csv("delay_test.csv")
prems = pd.read_csv("Premiums.csv")

In [11]:
delaytriangle(delay_test)


{'Table':    Years    0           1           2           3
 0   2005   96  136.000000  140.000000  168.000000
 1   2006  100  156.000000  160.000000  192.000000
 2   2007  120  130.000000  133.561644  160.273973
 3   2008  136  181.620253  186.596151  223.915381,
 'Total_Claims': 744.189353216577,
 'Settled_Claims': 594.0,
 'Required_Reserves': 150.18935321657705}

In [12]:
bftriangle(delay_test, prems)

{'Table':    Years    0           1           2           3
 0   2005   96  136.000000  140.000000  168.000000
 1   2006  100  156.000000  160.000000  192.000000
 2   2007  120  130.000000  133.561644  160.273973
 3   2008  136  181.620253  186.596151  223.915381,
 'Total_Claims': 744.189353216577,
 'Settled_Claims': 594.0,
 'Required_Reserves': 150.18935321657705,
 'Premiums':    Premiums Received
 0                175
 1                181
 2                190
 3                196,
 'Ultimate_Claims':    Premiums Received
 0             168.00
 1             173.76
 2             182.40
 3             188.16,
 'Emerging_Liabilities': [0.0,
  28.959999999999994,
  34.45333333333333,
  73.87682780410744],
 'Overall_Reserve_Requirement': 137.29016113744075}